In [ ]:
import pandas as pd
import re
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import networkx as nx

In [ ]:
#Esse código realiza as seguintes etapas:
#Limpeza de texto com remoção de stop words
#Análise de frequência para identificar as palavras mais comuns
#Geração de nuvem de palavras para visualização das frequências
#Agrupamento de respostas similares (análise de clusters) com KMeans
#Análise de similitude e criação de redes semânticas com base nas co-ocorrências ajustadas.
#Cada etapa foi personalizada para simplificar a análise e realçar os principais padrões nas respostas.

# Função de limpeza de texto com stop words adicionais
basic_stop_words = {
    "a", "e", "o", "que", "de", "do", "da", "em", "um", "uma", "para", "é", "no", "com", "por", "não", "como",
    "na", "se", "os", "ou", "ao", "dos", "as", "mas", "meu", "sua", "mais", "já", "mesmo", "assim", "qual",
    "nos", "também", "então", "isso", "seu”
}

def clean_text(text):
    # Lowercase, remove non-alphabetic characters, remove stop words
    text = re.sub(r'[^a-zA-ZáéíóúâêîôûãõçÁÉÍÓÚÂÊÎÔÛÃÕÇ\s]', '', text.lower())
    for stop_word in basic_stop_words:
        text = re.sub(r'\b' + stop_word + r'\b', '', text)
    return re.sub(r'\s+', ' ', text).strip()

In [ ]:
# Carregar e limpar os dados
file_path = '/mnt/data/respostas_sem_ident.xlsx'
data = pd.read_excel(file_path)
open_ended_columns = [
    "Fala pra gente um pouco da razão pela qual você quer fazer parte do programa cultura",
    "Expectativas. Por definição, expectativa é aquilo que a gente coloca no outro. Quais são suas expectativas em relação ao programa? O que o programa cultura pode fazer por você? O que você espera d...",
    "Intenção. Diferente da expectativa, intenção é aquilo que está em nós mesmos. Quais são suas intenções? O que você trará para o programa? Quais compromissos você assume?",
    "Pra você, o que é inovação?"
]
for col in open_ended_columns:
    data[col + "_cleaned"] = data[col].astype(str).apply(clean_text)

In [ ]:
# Análise de Frequência
word_frequencies = {col: Counter(' '.join(data[col + "_cleaned"]).split()) for col in open_ended_columns}
word_frequencies_summary = {col: freq.most_common(10) for col, freq in word_frequencies.items()}

In [ ]:
# Nuvem de Palavras
for col in open_ended_columns:
    wordcloud = WordCloud(width=800, height=400, background_color="white").generate_from_frequencies(word_frequencies[col])
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(f"Nuvem de Palavras: {col}", fontsize=14)
    plt.axis("off")
    plt.show()

In [ ]:
# Análise de Clusters
def cluster_responses(column_cleaned, num_clusters=5):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(data[column_cleaned].dropna())
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(X)
    return kmeans.labels_

for col in open_ended_columns:
    data[col + "_cluster"] = cluster_responses(col + "_cleaned")

In [ ]:
# Análise de Similitude e Rede Semântica Ajustada
higher_threshold = 0.3
for col in open_ended_columns:
    vectorizer = TfidfVectorizer(max_features=50)  # Limita a 50 termos
    X = vectorizer.fit_transform(data[col + "_cleaned"].dropna())
    co_occurrence_matrix = cosine_similarity(X.T)
    terms = vectorizer.get_feature_names_out()
    
    # Construir rede semântica
    G = nx.Graph()
    for i in range(len(terms)):
        for j in range(i + 1, len(terms)):
            if co_occurrence_matrix[i, j] > higher_threshold:
                G.add_edge(terms[i], terms[j], weight=co_occurrence_matrix[i, j])
    
    # Exibir a rede
    plt.figure(figsize=(10, 10))
    pos = nx.spring_layout(G, k=0.5)
    nx.draw(G, pos, with_labels=True, node_size=20, font_size=10, edge_color="gray")
    plt.title(f"Rede Semântica Ajustada: {col}", fontsize=14)
    plt.show()